In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (train_test_split,
                                     cross_val_score)
from sklearn.metrics import (accuracy_score, 
                             precision_score, 
                             recall_score)
import pprint
import keras
import tensorflow as tf
from keras.utils import np_utils
from keras.models import (Model,
                          Sequential)
from keras.layers import (Input, 
                          Flatten, 
                          Dropout, 
                          Activation, 
                          Dense, 
                          Conv1D, 
                          MaxPooling1D)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

print(f'keras version = {keras.__version__}, tensorflow version = {tf.__version__}')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


keras version = 2.4.3, tensorflow version = 2.3.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**9. Classification Models**

##**9.1 Random Forest Classification**

**Load df_embeddings_pca data**

In [ ]:
df_embeddings_pca = pd.read_csv(os.path.join(path, 'df_embeddings_pca.csv'), index_col=False)
df_embeddings_pca.shape

(305520, 62)

In [ ]:
df_embeddings_pca.head(3)

,Unnamed: 0,site_id,well_id,cell_type,experiment,plate,well,site,disease_condition,treatment,treatment_conc,SMILES,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,HRCE-1_1_AA02_1,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,1,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...,0.132417,0.319708,0.316053,0.391044,0.393467,0.590292,0.191758,0.351341,0.461592,0.466987,0.536253,0.437953,0.572890,0.391879,0.349071,0.445347,0.341852,0.504132,0.604077,0.399611,0.427331,0.464915,0.465775,0.489664,0.494247,0.408223,0.506484,0.453152,0.484760,0.495087,0.524508,0.509021,0.401143,0.385463,0.445719,0.508025,0.490813,0.481652,0.421074,0.465967,0.453822,0.464899,0.369199,0.442958,0.382452,0.431862,0.404141,0.463700,0.492772,0.450314
1,1,HRCE-1_1_AA02_2,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,2,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...,0.061210,0.292411,0.343654,0.439015,0.361604,0.531461,0.252884,0.412200,0.441596,0.501719,0.495689,0.523878,0.535313,0.316499,0.323712,0.450942,0.355204,0.422611,0.508067,0.463249,0.329443,0.415455,0.519154,0.433636,0.505194,0.399102,0.473974,0.459359,0.443502,0.536214,0.593173,0.437939,0.304154,0.428820,0.418285,0.410625,0.586705,0.457273,0.424965,0.500659,0.387446,0.472137,0.363910,0.413070,0.348539,0.460638,0.422661,0.456728,0.496217,0.373741
2,2,HRCE-1_1_AA02_3,HRCE-1_1_AA02,HRCE,HRCE-1,1,AA02,3,Active SARS-CoV-2,Flubendazole,0.1,COC(=O)NC1=NC2=C(N1)C=C(C=C2)C(=O)C1=CC=C(F)C=...,0.108674,0.336356,0.406311,0.407748,0.389359,0.544734,0.170589,0.417111,0.398482,0.501507,0.543979,0.426225,0.562843,0.373681,0.286814,0.442930,0.384244,0.447445,0.487311,0.343914,0.411653,0.455362,0.475752,0.472554,0.568790,0.421586,0.461438,0.463248,0.488274,0.551369,0.478059,0.458264,0.422222,0.409242,0.461232,0.479216,0.528342,0.401276,0.392861,0.525454,0.422534,0.483994,0.356430,0.436135,0.413837,0.474573,0.408615,0.431241,0.461833,0.365500


In [ ]:
df_embeddings_pca = df_embeddings_pca[df_embeddings_pca['disease_condition'].notnull()]
y = df_embeddings_pca.pop('disease_condition')
X = df_embeddings_pca[['feature_' + str(i) for i in range(50)]]

In [ ]:
print(X.shape, type(X))
print(y.shape, type(y))

(298616, 50) <class 'pandas.core.frame.DataFrame'>
(298616,) <class 'pandas.core.series.Series'>


**Random forest model**

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
%%time
model = rnd_clf.fit(X_train, y_train)

CPU times: user 33min 21s, sys: 1.42 s, total: 33min 22s
Wall time: 8min 26s


In [ ]:
%%time
y_train_pred = model.predict(X_train)
y_train_pred.shape

CPU times: user 37.5 s, sys: 215 ms, total: 37.8 s
Wall time: 9.76 s


In [ ]:
%%time
y_test_pred = model.predict(X_test)
print(y_test_pred.shape)

(59724,)
CPU times: user 9.62 s, sys: 123 ms, total: 9.74 s
Wall time: 2.62 s


**Summary output**

In [ ]:
output = {
    'model': 'Random Forest',
    'accracy_train': accuracy_score(y_train, y_train_pred),
    'accuracy_test': accuracy_score(y_test, y_test_pred),
    'precision_test': precision_score(y_test, y_test_pred, average='weighted'),
    'recall_test': recall_score(y_test, y_test_pred, average='weighted')
}

pprint.pprint(f'output = {output}')

("output = {'model': 'Random Forest', 'accracy_train': 1.0, 'accuracy_test': "
 "0.9647545375393477, 'precision_test': 0.9598040730702959, 'recall_test': "
 '0.9647545375393477}')


##**9.2. Convolutional Neural Network**  
**using 1024 features of embeddings subset dataset (9120 rows for each disease condition)**

In [ ]:
df_embeddings_subset = pd.read_csv(os.path.join(path, 'embedddings_subset.csv'), index_col=False)
df_embeddings_subset.shape

(27360, 1025)

In [ ]:
df_embeddings_subset.head(3)

,site_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,...,feature_984,feature_985,feature_986,feature_987,feature_988,feature_989,feature_990,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,feature_1000,feature_1001,feature_1002,feature_1003,feature_1004,feature_1005,feature_1006,feature_1007,feature_1008,feature_1009,feature_1010,feature_1011,feature_1012,feature_1013,feature_1014,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023
0,HRCE-1_10_AA05_1,2.389372,0.214582,-0.272902,-0.196034,-0.563311,0.619960,-0.371054,-0.037861,-0.219545,-0.392556,-2.769379,-0.389006,-0.543331,1.143736,-0.567993,-0.919517,0.131801,-0.072684,0.043436,-0.041997,-0.598043,-0.256680,0.499137,0.038466,-2.324510,0.587085,0.148490,-1.344135,0.615269,1.145046,0.951628,-0.165207,-1.689007,0.681347,-1.732188,0.408793,-1.773126,0.600478,-1.289480,...,1.694329,-0.702744,0.300283,0.360717,0.096897,-0.119458,0.089974,0.388082,-1.095558,0.061971,-0.69445,-0.470560,0.165727,0.954380,0.641269,0.000110,1.180884,0.871668,0.718739,0.122943,1.184700,0.337163,-1.053558,-0.368507,-0.772576,1.022397,0.306467,0.005132,-2.340652,0.668188,1.172451,0.989767,-1.486712,-1.447653,-1.304442,-0.031610,-0.710898,0.555251,-0.265755,-1.212434
1,HRCE-1_10_AA08_1,2.297975,0.022587,0.195894,0.014781,-1.100471,0.098462,0.242965,0.199119,-0.567304,-0.227210,-2.230506,-0.949541,-0.047310,0.533605,-0.290319,-1.508357,0.161617,0.380366,0.146494,-0.108530,-0.873874,0.056584,0.491351,0.080825,-1.321582,0.298009,0.190170,-0.864164,0.351424,0.536975,1.282919,-0.110490,-1.274246,0.943124,-1.541825,0.065526,-1.234762,0.295345,-1.824088,...,1.209590,-1.098585,0.753858,0.079735,0.000902,-0.457200,-0.152898,0.280296,-0.768077,0.315934,-0.38891,-0.073659,0.390859,0.923492,0.642634,0.143817,0.619918,0.375651,0.347446,-0.236212,1.128898,0.012419,-1.073843,-0.257170,-0.714517,0.858910,0.012951,-0.503524,-1.609319,1.182596,0.586100,0.781158,-1.187773,-1.368313,-1.659620,-0.020060,-0.498513,0.349791,-0.286453,-1.424416
2,HRCE-1_10_AA08_2,2.023117,0.055359,0.032669,-0.427921,-1.477027,0.555283,0.125149,0.024121,-0.310992,-0.035899,-2.317528,-0.571142,-0.190510,1.388795,-0.545442,-1.311690,0.067700,0.051449,0.281658,0.190968,-0.541562,0.122109,0.277549,-0.053468,-1.983142,0.453062,0.017778,-1.351487,0.600766,1.270639,1.074047,-0.062500,-1.360750,0.680048,-1.492564,0.104654,-1.461042,0.506609,-1.586523,...,1.547907,-0.983013,0.355730,0.020508,0.567637,-0.209212,-0.159852,0.014123,-0.657308,0.370420,-0.90858,-0.119487,0.233618,0.823882,0.551085,-0.014573,1.159965,0.614649,0.434470,0.093707,1.066292,0.126574,-1.049656,-0.243583,-0.585675,0.663529,-0.127107,0.177727,-1.868272,1.141989,0.857226,0.687854,-1.667390,-1.059504,-1.597720,-0.335833,-0.643573,0.253038,0.145723,-1.794530


In [ ]:
df_embeddings_subset.set_index('site_id', inplace=True)

In [ ]:
df_embeddings_subset.head(3)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_984,feature_985,feature_986,feature_987,feature_988,feature_989,feature_990,feature_991,feature_992,feature_993,feature_994,feature_995,feature_996,feature_997,feature_998,feature_999,feature_1000,feature_1001,feature_1002,feature_1003,feature_1004,feature_1005,feature_1006,feature_1007,feature_1008,feature_1009,feature_1010,feature_1011,feature_1012,feature_1013,feature_1014,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023
site_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HRCE-1_10_AA05_1,2.389372,0.214582,-0.272902,-0.196034,-0.563311,0.619960,-0.371054,-0.037861,-0.219545,-0.392556,-2.769379,-0.389006,-0.543331,1.143736,-0.567993,-0.919517,0.131801,-0.072684,0.043436,-0.041997,-0.598043,-0.256680,0.499137,0.038466,-2.324510,0.587085,0.148490,-1.344135,0.615269,1.145046,0.951628,-0.165207,-1.689007,0.681347,-1.732188,0.408793,-1.773126,0.600478,-1.289480,1.741578,...,1.694329,-0.702744,0.300283,0.360717,0.096897,-0.119458,0.089974,0.388082,-1.095558,0.061971,-0.69445,-0.470560,0.165727,0.954380,0.641269,0.000110,1.180884,0.871668,0.718739,0.122943,1.184700,0.337163,-1.053558,-0.368507,-0.772576,1.022397,0.306467,0.005132,-2.340652,0.668188,1.172451,0.989767,-1.486712,-1.447653,-1.304442,-0.031610,-0.710898,0.555251,-0.265755,-1.212434
HRCE-1_10_AA08_1,2.297975,0.022587,0.195894,0.014781,-1.100471,0.098462,0.242965,0.199119,-0.567304,-0.227210,-2.230506,-0.949541,-0.047310,0.533605,-0.290319,-1.508357,0.161617,0.380366,0.146494,-0.108530,-0.873874,0.056584,0.491351,0.080825,-1.321582,0.298009,0.190170,-0.864164,0.351424,0.536975,1.282919,-0.110490,-1.274246,0.943124,-1.541825,0.065526,-1.234762,0.295345,-1.824088,1.225561,...,1.209590,-1.098585,0.753858,0.079735,0.000902,-0.457200,-0.152898,0.280296,-0.768077,0.315934,-0.38891,-0.073659,0.390859,0.923492,0.642634,0.143817,0.619918,0.375651,0.347446,-0.236212,1.128898,0.012419,-1.073843,-0.257170,-0.714517,0.858910,0.012951,-0.503524,-1.609319,1.182596,0.586100,0.781158,-1.187773,-1.368313,-1.659620,-0.020060,-0.498513,0.349791,-0.286453,-1.424416
HRCE-1_10_AA08_2,2.023117,0.055359,0.032669,-0.427921,-1.477027,0.555283,0.125149,0.024121,-0.310992,-0.035899,-2.317528,-0.571142,-0.190510,1.388795,-0.545442,-1.311690,0.067700,0.051449,0.281658,0.190968,-0.541562,0.122109,0.277549,-0.053468,-1.983142,0.453062,0.017778,-1.351487,0.600766,1.270639,1.074047,-0.062500,-1.360750,0.680048,-1.492564,0.104654,-1.461042,0.506609,-1.586523,1.265087,...,1.547907,-0.983013,0.355730,0.020508,0.567637,-0.209212,-0.159852,0.014123,-0.657308,0.370420,-0.90858,-0.119487,0.233618,0.823882,0.551085,-0.014573,1.159965,0.614649,0.434470,0.093707,1.066292,0.126574,-1.049656,-0.243583,-0.585675,0.663529,-0.127107,0.177727,-1.868272,1.141989,0.857226,0.687854,-1.667390,-1.059504,-1.597720,-0.335833,-0.643573,0.253038,0.145723,-1.794530


In [ ]:
df_meta_subset = pd.read_csv(os.path.join(path, 'meta_subset.csv'), index_col=False)
df_meta_subset.shape

(27360, 11)

In [ ]:
# target is disease condition
target = df_meta_subset['disease_condition']
print(type(target), len(target))

<class 'pandas.core.series.Series'> 27360


**Data preparation**

In [ ]:
# Reshape dimension of embedding_subset from 2D to 3D since CNN require 3D input
data = np.expand_dims(df_embeddings_subset, axis=2)

In [ ]:
data.shape

(27360, 1024, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
# print(type(y_train), type(y_test))
# Convert pandas series of label to np array
y_train_ = np.array(pd.factorize(y_train)[0])
y_test_ = np.array(pd.factorize(y_test)[0])
print(y_train_[0:100])

[0 0 1 0 0 0 0 0 1 2 0 1 1 2 2 0 1 1 2 1 2 0 2 1 1 2 0 0 2 2 1 1 1 1 1 2 2
 2 0 0 2 1 2 2 1 1 0 2 2 1 1 0 2 1 2 1 2 1 2 0 1 0 0 1 0 0 0 0 2 2 0 1 2 2
 2 2 2 0 0 2 1 2 1 1 2 2 2 1 1 2 2 2 2 1 0 2 2 1 0 2]


In [ ]:
# Convert label numpy array to on-hot vector
y_train = np_utils.to_categorical(y_train_)
y_test = np_utils.to_categorical(y_test_)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(21888, 1024, 1) (5472, 1024, 1) (21888, 3) (5472, 3)


In [ ]:
print(np.array(y_test).shape)

(5472, 3)


**A CNN model (An example model for test)**

In [ ]:
%%time
dropout = 0.1

model = Sequential()

model.add(Conv1D(64, 3, padding='same', activation='relu', input_shape=(1024, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(128, 3, padding='same', activation='relu'))
model.add(Conv1D(128, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(256, 3, padding='same', activation='relu'))
model.add(Conv1D(256, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(64, activation='relu'))
model.add(Dropout(dropout))

model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

cnn = model.fit(X_train, y_train, 
                batch_size=32, 
                epochs=50, 
                validation_data=(X_test, y_test))

Epoch 1/50
684/684 [==============================] - 202s 296ms/step - loss: 0.5495 - accuracy: 0.7286 - val_loss: 0.4220 - val_accuracy: 0.7990
Epoch 2/50
684/684 [==============================] - 204s 299ms/step - loss: 0.4109 - accuracy: 0.8073 - val_loss: 0.3762 - val_accuracy: 0.8198
Epoch 3/50
684/684 [==============================] - 202s 296ms/step - loss: 0.3773 - accuracy: 0.8271 - val_loss: 0.4150 - val_accuracy: 0.7984
Epoch 4/50
684/684 [==============================] - 206s 302ms/step - loss: 0.3379 - accuracy: 0.8475 - val_loss: 0.3626 - val_accuracy: 0.8377
Epoch 5/50
684/684 [==============================] - 203s 296ms/step - loss: 0.3197 - accuracy: 0.8572 - val_loss: 0.3498 - val_accuracy: 0.8397
Epoch 6/50
684/684 [==============================] - 203s 297ms/step - loss: 0.2998 - accuracy: 0.8701 - val_loss: 0.3184 - val_accuracy: 0.8540
Epoch 7/50
684/684 [==============================] - 204s 298ms/step - loss: 0.2857 - accuracy: 0.8743 - val_loss: 0.3337 -

**Note: epochs = 40 is enough**

In [ ]:
print(f'Accuracy = {cnn.history["accuracy"][-1]*100}')

Accuracy = 98.34612607955933


In [ ]:
print(f'Validation Accuracy = {cnn.history["val_accuracy"][-1]*100}')

Validation Accuracy = 87.17105388641357


In [ ]:
%%time
y_train_pred = model.predict_classes(X_train)
y_test_pred = model.predict_classes(X_test)
print(y_train_pred.shape, y_test_pred.shape)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
(21888,) (5472,)
CPU times: user 3min 35s, sys: 2.05 s, total: 3min 37s
Wall time: 1min 4s


In [ ]:
print(type(y_train_pred), y_train_pred.shape, y_train_pred[0:5])
print(type(y_train_), y_train_.shape, y_train_[0:5])

<class 'numpy.ndarray'> (21888,) [0 0 1 0 0]
<class 'numpy.ndarray'> (21888,) [0 0 1 0 0]


**Summary**

In [ ]:
output = {
    'model': 'CNN',
    'accracy_train': accuracy_score(y_train_, y_train_pred),
    'accuracy_test': accuracy_score(y_test_, y_test_pred),
    'precision_test': precision_score(y_test_, y_test_pred, average='weighted'),
    'recall_test': recall_score(y_test_, y_test_pred, average='weighted')
}

pprint.pprint(f'output = {output}')

("output = {'model': 'CNN', 'accracy_train': 0.9975785818713451, "
 "'accuracy_test': 0.8717105263157895, 'precision_test': 0.8718082058252381, "
 "'recall_test': 0.8717105263157895}")


**As illustrated above, it is possible to predict the disease conditions with a high accuracy. Of course, CNN model needs to be modified and tunning**

**The next important step is to predict the efficacy of each drug (compound) with the different doses (concentrations)**  


**End of Part 4.**